## Dependencies

In [1]:
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
model_path = '/kaggle/input/104-googleq-a-train-bert-base-raw/model.h5'
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
TOKENIZER_PATH = BERT_PATH+'/assets/vocab.txt'

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(TOKENIZER_PATH, True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

bert_layer = KerasLayer(BERT_PATH, trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

x = GlobalAveragePooling1D()(sequence_output)
x = Dropout(0.2)(x)
output = Dense(N_CLASS, kernel_initializer='glorot_uniform', activation="sigmoid", name="output")(x)

model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)

model.summary()

model.load_weights(model_path)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

# Make predictions

In [8]:
Y_test = model.predict(X_test)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.934982,0.521750,0.332356,0.370878,0.437367,0.597576,0.629542,0.567304,0.634499,...,0.903181,0.889945,0.489035,0.967594,0.941745,0.821764,0.065981,0.046886,0.807168,0.914318
1,46,0.896119,0.472068,0.008388,0.696288,0.794810,0.908436,0.531180,0.433234,0.033802,...,0.701592,0.942678,0.628149,0.969624,0.983214,0.842396,0.958196,0.096975,0.044063,0.883652
2,70,0.924544,0.600702,0.062681,0.735397,0.867726,0.944151,0.567220,0.478873,0.089925,...,0.840666,0.904981,0.630470,0.976117,0.959884,0.834777,0.052915,0.047914,0.863309,0.925134
3,132,0.891267,0.400911,0.012442,0.653621,0.767781,0.918779,0.535990,0.439204,0.075596,...,0.702950,0.924845,0.673489,0.967341,0.983548,0.863941,0.894820,0.147173,0.295025,0.878204
4,200,0.906302,0.335389,0.116654,0.636905,0.632643,0.748930,0.553880,0.500133,0.089760,...,0.622123,0.843455,0.583944,0.961364,0.948913,0.786490,0.145695,0.100599,0.517030,0.922407


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.903251,0.540494,0.047761,0.673037,0.748074,0.846731,0.559690,0.452088,0.219165,...,0.799108,0.913320,0.652057,0.963586,0.970347,0.842512,0.561289,0.121897,0.473419,0.910372
std,2812.670060,0.044065,0.130073,0.076165,0.128920,0.124921,0.143412,0.052604,0.082115,0.187145,...,0.082423,0.030705,0.052820,0.012461,0.013736,0.045005,0.344270,0.062797,0.288460,0.023053
min,39.000000,0.708853,0.274356,0.004491,0.230164,0.176599,0.225015,0.418605,0.294217,0.005390,...,0.604492,0.787264,0.470838,0.916507,0.907875,0.708453,0.006251,0.007478,0.009560,0.799225
25%,2572.000000,0.879769,0.425773,0.011010,0.600397,0.712526,0.823090,0.522430,0.396636,0.078842,...,0.732382,0.895925,0.619200,0.956683,0.963662,0.817926,0.192928,0.075849,0.208944,0.895522
50%,5093.000000,0.909305,0.524075,0.017437,0.680337,0.768371,0.901437,0.554124,0.425675,0.147332,...,0.796905,0.916301,0.651440,0.965076,0.973635,0.845420,0.680049,0.115130,0.447885,0.911216
75%,7482.000000,0.935612,0.644562,0.043117,0.749658,0.818777,0.933565,0.592273,0.498151,0.332887,...,0.869921,0.934084,0.684808,0.971745,0.980012,0.873861,0.875855,0.161880,0.705046,0.925585
max,9640.000000,0.979039,0.901776,0.570008,0.976003,0.967134,0.987446,0.736788,0.728027,0.779602,...,0.950994,0.976569,0.798165,0.992815,0.991823,0.954073,0.984234,0.311900,0.984323,0.967129
